Hi Buddy!<br> well for this exercise we will be following an <a ref='https://www.kdnuggets.com/guide-deploying-machine-learning-models-production'>article</a>.

First open your terminal and create an virtual environment.
> python -m venv myvirtualenv

Create the `requirements.txt` file and fill it out with the following library list.
<br>

- pandas
- scikit-learn
- fastapi
- pydantic
- uvicorn
- streamlit


We must install them using the following code.
> pip install -r requirements.txt

Using the kaggle dataset,<br>
https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
<br> we will be training the model.

#### Train the model

Create a file called `app/train_model.py` in the app folder. 
Write this code.

Create an empty folder, `models`

In [ ]:
import pandas as pd
import joblib   # for saving the model
from sklearn.linear_model import LogisticRegression

data = pd.read_csv("data\\diabetes.csv")
X = data.drop('Outcome', axis =1)
y = data['Outcome']
model = LogisticRegression()

model.fit(X, y)
joblib.dump(model, 'models\\logreg_model.joblib')

#### **Model Deployment**

We will create API for our model prediction and deploy them with Docker while testing them with the Streamlit front end.
First, ensure you already have a docker desktop installed, as we will test them locally.

!!! Create a file called `app/main.py` in the app folder and fill it with the following code to generate the API.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib

# Load the logistic regression model
model = joblib.load('../models/logreg_model.joblib')

# Define the input data model
class DiabetesData(BaseModel):
    Pregnancies: int
    Glucose: int
    BloodPressure: int
    SkinThickness: int
    Insulin: int
    BMI: float
    DiabetesPedigreeFunction: float
    Age: int
app = FastAPI()

# Define prediction endpoint
@app.post("/predict")
def predict(data: DiabetesData):
    input_data = {
        'Pregnancies': [data.Pregnancies],
        'Glucose': [data.Glucose],
        'BloodPressure': [data.BloodPressure],
        'SkinThickness': [data.SkinThickness],
        'Insulin': [data.Insulin],
        'BMI': [data.BMI],
        'DiabetesPedigreeFunction': [data.DiabetesPedigreeFunction],
        'Age': [data.Age]
    }
    input_df = pd.DataFrame(input_data)

    # Make a prediction
    prediction = model.predict(input_df)
    result = "Diabetes" if prediction[0] == 1 else "Not Diabetes"
    return {"prediction": result}

Additionally, we will have a frontend web to try the API model we deployed.
To do that, create a file called `app/frontend.py` in the app folder. Then, fill them with the following code.

To RUN streamlit, always use in terminal
> streamlit run frontend.py


In [ ]:
import streamlit as st
import requests
import json

API_URL = "http://localhost:8000/predict"

st.title("Diabetes Prediction App")
st.write("Enter the details below to make a prediction.")

pregnancies = st.number_input("Pregnancies", min_value=0, step=1)
glucose = st.number_input("Glucose", min_value=0, step=1)
blood_pressure = st.number_input("Blood Pressure", min_value=0, step=1)
skin_thickness = st.number_input("Skin Thickness", min_value=0, step=1)
insulin = st.number_input("Insulin", min_value=0, step=1)
bmi = st.number_input("BMI", min_value=0.0, step=0.1)
diabetes_pedigree_function = st.number_input("Diabetes Pedigree Function", min_value=0.0, step=0.1)
age = st.number_input("Age", min_value=0, step=1)

if st.button("Predict"):
    input_data = {
        "Pregnancies": pregnancies,
        "Glucose": glucose,
        "BloodPressure": blood_pressure,
        "SkinThickness": skin_thickness,
        "Insulin": insulin,
        "BMI": bmi,
        "DiabetesPedigreeFunction": diabetes_pedigree_function,
        "Age": age
    }

    response = requests.post(API_URL, data=json.dumps(input_data), headers={"Content-Type": "application/json"})
   
    if response.status_code == 200:
        prediction = response.json().get("prediction", "No prediction")
        st.success(f"Prediction: {prediction}")
    else:
        st.error("Error in making prediction. Please check your input data and try again.")
 

When everything is ready, we will create the `Dockerfile` as the basis for our model deployment. 

> You should fill in the code below in the file.

FROM python:3.9-slim

WORKDIR /app

COPY app /app
COPY models /models

RUN pip install --no-cache-dir --upgrade pip && \
    pip install --no-cache-dir -r requirements.txt

EXPOSE 8000 8501

CMD ["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port 8000 & streamlit run frontend.py --server.port=8501 --server.enableCORS=false"]

We will create the image with the Docker file ready and then deploy the model via container.
RUN IN TERMINAL
> docker build -t diabetes-prediction-app .

The code above creates the Docker image for our model container. Then, we will use the following code to make the API for model deployment.
> docker run -d -p 8000:8000 -p 8501:8501 --name diabetes-prediction-container diabetes-prediction-app

With everything ready, ensure the container runs and access the front end with the address below.
> http://localhost:8501/